<a href="https://colab.research.google.com/github/srinayani123/Mentalhealth_reddit_classification/blob/main/Model_finetuning/classification/roberta_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install dependencies
!pip install transformers datasets evaluate -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import (
    RobertaConfig,
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer
)
import evaluate

In [ ]:
# 1. Load and clean your data
from google.colab import files
import io

uploaded = files.upload()
filename = list(uploaded.keys())[0]  # get uploaded file name
df = pd.read_csv(io.BytesIO(uploaded[filename]))

df["text"] = df["title"].fillna("") + " " + df["text"].fillna("")
df["label"] = df["target"]
df = df[["text", "label"]]


Saving data_to_be_cleansed.csv to data_to_be_cleansed.csv


In [ ]:
# 2. Train-test split
train_df, test_df = train_test_split(
    df,
    test_size=0.2,
    stratify=df["label"],
    random_state=42
)

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)


In [ ]:
# 3. Initializing tokenizer (standard RoBERTa tokenizer)
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
# 4. Tokenize examples
def tokenize_function(batch):
    return tokenizer(batch["text"], truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/4765 [00:00<?, ? examples/s]

Map:   0%|          | 0/1192 [00:00<?, ? examples/s]

In [ ]:
# 5. Create custom RoBERTa config for training from scratch
config = RobertaConfig(
    vocab_size=tokenizer.vocab_size,
    hidden_size=768,
    num_attention_heads=12,
    num_hidden_layers=6,  # fewer layers to train faster
    intermediate_size=3072,
    max_position_embeddings=514,
    num_labels=5
)


In [ ]:
model_r = RobertaForSequenceClassification(config=config)

In [ ]:
# 7. Data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
# 8. Define accuracy metric
accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return accuracy_metric.compute(predictions=preds, references=labels)


In [ ]:
# 9. Training arguments
training_args = TrainingArguments(
    output_dir="./roberta-results",
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=3e-5,
    weight_decay=0.01,
    logging_dir="./logs-roberta",
    logging_steps=100
)


In [ ]:
# 10. Trainer
trainer = Trainer(
    model=model_r,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

/tmp/ipython-input-12-101456195.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# 11. Train the model
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mankali-srinayani (mankali-srinayani-other) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
100,1.703000
200,1.653200
300,1.640300
400,1.637100
500,1.517600
600,1.436300
700,1.359400
800,1.286500
900,1.209600
1000,1.116700


TrainOutput(global_step=2980, training_loss=0.8316081738311972, metrics={'train_runtime': 1513.6932, 'train_samples_per_second': 15.74, 'train_steps_per_second': 1.969, 'total_flos': 2575226122666170.0, 'train_loss': 0.8316081738311972, 'epoch': 5.0})

In [ ]:
trainer.save_model("./final_roberta_scratch_model")
tokenizer.save_pretrained("./final_roberta_scratch_model")


('./final_roberta_scratch_model/tokenizer_config.json',
 './final_roberta_scratch_model/special_tokens_map.json',
 './final_roberta_scratch_model/vocab.json',
 './final_roberta_scratch_model/merges.txt',
 './final_roberta_scratch_model/added_tokens.json',
 './final_roberta_scratch_model/tokenizer.json')

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding
import torch
import numpy as np


In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
test_dataloader = DataLoader(
    test_dataset.remove_columns(["text", "__index_level_0__"]) if "__index_level_0__" in test_dataset.column_names else test_dataset.remove_columns(["text"]),
    batch_size=16,
    collate_fn=data_collator
)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_r.to(device)
model_r.eval()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
           

In [ ]:
# Evaluate in batches
all_preds = []
all_labels = []

for batch in test_dataloader:
    labels = batch["labels"].to(device)
    inputs = {k: v.to(device) for k, v in batch.items() if k != "labels"}

    with torch.no_grad():
        outputs = model_r(**inputs)
    logits = outputs.logits
    preds = torch.argmax(logits, axis=1)

    all_preds.extend(preds.cpu().numpy())
    all_labels.extend(labels.cpu().numpy())


In [ ]:
# Evaluation metrics
accuracy = accuracy_score(all_labels, all_preds)
precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average="weighted")

print("\n📊 Evaluation Report:")
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1:.4f}")

print("\n📋 Classification Report (per class):")
print(classification_report(all_labels, all_preds, digits=4))


📊 Evaluation Report:
Accuracy : 0.7794
Precision: 0.7802
Recall   : 0.7794
F1 Score : 0.7781

📋 Classification Report (per class):
              precision    recall  f1-score   support

           0     0.8167    0.8686    0.8419       236
           1     0.8060    0.6722    0.7330       241
           2     0.7698    0.8186    0.7935       237
           3     0.7574    0.7417    0.7495       240
           4     0.7510    0.7983    0.7739       238

    accuracy                         0.7794      1192
   macro avg     0.7802    0.7799    0.7784      1192
weighted avg     0.7802    0.7794    0.7781      1192



In [ ]:
true_labels = all_labels
pred_labels = all_preds


In [ ]:
import plotly.express as px
import pandas as pd
from sklearn.metrics import classification_report


true_labels = [int(label) for label in true_labels]
pred_labels = [int(label) for label in pred_labels]

# Generate classification report
report = classification_report(true_labels, pred_labels, output_dict=True, zero_division=0)
report_df = pd.DataFrame(report).transpose().reset_index()
report_df = report_df[report_df["index"].isin(["0", "1", "2", "3", "4"])]
report_df.rename(columns={"index": "Class", "precision": "Precision", "recall": "Recall", "f1-score": "F1 Score"}, inplace=True)
report_df = report_df[["Class", "Precision", "Recall", "F1 Score"]]

# Melt for Plotly
melted_df = report_df.melt(id_vars="Class", var_name="Metric", value_name="Score")

# Plot
fig = px.bar(
    melted_df,
    x="Metric",
    y="Score",
    color="Class",
    barmode="group",
    facet_col="Class",
    text="Score",
    title="📊 Evaluation Metrics Per Class",
    color_discrete_sequence=px.colors.sequential.Plasma,
    template="plotly_dark",
    height=500
)

fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_layout(
    paper_bgcolor='black',
    plot_bgcolor='black',
    font=dict(color='white'),
    title_font=dict(size=20),
    uniformtext_minsize=8,
    uniformtext_mode='hide',
    xaxis_title="Evaluation Metric",
    yaxis_title="Score",
    showlegend=False
)

fig.show()


In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.metrics import confusion_matrix


# Compute confusion matrix
cm = confusion_matrix(true_labels, pred_labels)
cm_df = pd.DataFrame(cm, index=[f"True {i}" for i in range(cm.shape[0])],
                        columns=[f"Pred {i}" for i in range(cm.shape[1])])

# Plot confusion matrix
fig_cm = px.imshow(
    cm_df,
    text_auto=True,
    color_continuous_scale="Plasma",
    title="📊 Confusion Matrix",
    aspect="auto"
)

fig_cm.update_layout(
    template="plotly_dark",
    paper_bgcolor='black',
    plot_bgcolor='black',
    font=dict(color='white'),
    title_font=dict(size=20),
    xaxis_title="Predicted Label",
    yaxis_title="True Label"
)

fig_cm.show()


In [ ]:
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
import plotly.graph_objects as go

# Convert labels to one-hot format
n_classes = len(np.unique(true_labels))
y_true_bin = label_binarize(true_labels, classes=list(range(n_classes)))
y_pred_bin = label_binarize(pred_labels, classes=list(range(n_classes)))

# Compute ROC curves
fig_roc = go.Figure()

for i in range(n_classes):
    fpr, tpr, _ = roc_curve(y_true_bin[:, i], y_pred_bin[:, i])
    roc_auc = auc(fpr, tpr)

    fig_roc.add_trace(go.Scatter(
        x=fpr, y=tpr,
        mode='lines',
        name=f'Class {i} (AUC={roc_auc:.2f})'
    ))

# Add random baseline
fig_roc.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', line=dict(dash='dash'), name='Random'))

# Layout
fig_roc.update_layout(
    title="📈 ROC Curves (One-vs-Rest)",
    xaxis_title="False Positive Rate",
    yaxis_title="True Positive Rate",
    template="plotly_dark",
    paper_bgcolor='black',
    plot_bgcolor='black',
    font=dict(color='white'),
    title_font=dict(size=20),
    height=500
)

fig_roc.show()


In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.metrics import confusion_matrix

# Compute confusion matrix
cm = confusion_matrix(true_labels, pred_labels)

# Zero diagonal to focus only on errors
cm_no_diag = cm.copy()
np.fill_diagonal(cm_no_diag, 0)

# Convert to DataFrame
cm_df = pd.DataFrame(cm_no_diag, index=[f"True {i}" for i in range(cm.shape[0])],
                                    columns=[f"Pred {i}" for i in range(cm.shape[1])])

# Plot error-only heatmap
fig_misclass = px.imshow(
    cm_df,
    text_auto=True,
    color_continuous_scale="Reds",
    title="🚨 Misclassification Heatmap",
    labels=dict(x="Predicted", y="Actual", color="Count")
)

fig_misclass.update_layout(
    template="plotly_dark",
    paper_bgcolor='black',
    plot_bgcolor='black',
    font=dict(color='white'),
    title_font=dict(size=20)
)

fig_misclass.show()


#SAMPLE INPUT OUTPUT

In [ ]:
import torch

# Sample Reddit-style posts
sample_texts = [
    "I can't focus at work, my chest feels tight and I'm constantly anxious.",
    "I’ve been crying every night and feel like I’m completely worthless.",
    "I stayed up for three nights straight and I feel unstoppable. Is this mania?",
    "Deadlines are crushing me. I’m drowning in work and I can't relax.",
    "My emotions swing wildly and I push people away even though I don't want to."
]

# Define the label mapping manually (as used during training)
id_to_label = {
    0: "Stress",
    1: "Depression",
    2: "Bipolar disorder",
    3: "Personality disorder",
    4: "Anxiety"
}

# Set device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_r.to(device)

# Tokenize and move to the same device
inputs = tokenizer(sample_texts, return_tensors="pt", padding=True, truncation=True)
inputs = {k: v.to(device) for k, v in inputs.items()}

# Predict
model_r.eval()
with torch.no_grad():
    outputs = model_r(**inputs)
    predictions = torch.argmax(outputs.logits, dim=1).tolist()

# Display predictions
for text, pred in zip(sample_texts, predictions):
    print(f"📝 Input:\n{text}\n🔍 Predicted Condition: {id_to_label[pred]}\n{'-'*60}")


📝 Input:
I can't focus at work, my chest feels tight and I'm constantly anxious.
🔍 Predicted Condition: Anxiety
------------------------------------------------------------
📝 Input:
I’ve been crying every night and feel like I’m completely worthless.
🔍 Predicted Condition: Depression
------------------------------------------------------------
📝 Input:
I stayed up for three nights straight and I feel unstoppable. Is this mania?
🔍 Predicted Condition: Bipolar disorder
------------------------------------------------------------
📝 Input:
Deadlines are crushing me. I’m drowning in work and I can't relax.
🔍 Predicted Condition: Depression
------------------------------------------------------------
📝 Input:
My emotions swing wildly and I push people away even though I don't want to.
🔍 Predicted Condition: Personality disorder
------------------------------------------------------------


#Roberta- regression- triage score prediction

In [ ]:
import re

In [ ]:
def compute_triage_score(text):
    text = text.lower()

    high_risk_patterns = [
        r"suicidal", r"kill myself", r"don’t want to live", r"ending it all", r"take my life",
        r"ending it", r"cried.*hours", r"can’t do this anymore", r"don’t matter", r"hate myself",
        r"thought about ending", r"disappear forever", r"can't keep going", r"stop existing",
        r"no reason to live", r"crying.*(ending it|ending everything)", r"gave up", r"worthless and hopeless"
    ]
    moderate_risk_patterns = [
        r"panic attack", r"shaking", r"can’t breathe", r"crying", r"racing thoughts",
        r"hopeless", r"worthless", r"empty", r"numb"
    ]
    mild_risk_patterns = [
        r"overwhelmed", r"can’t focus", r"tired", r"burnout", r"anxious", r"pressure", r"stressed"
    ]

    for pat in high_risk_patterns:
        if re.search(pat, text):
            return 1.0
    for pat in moderate_risk_patterns:
        if re.search(pat, text):
            return 0.75
    for pat in mild_risk_patterns:
        if re.search(pat, text):
            return 0.4
    return 0.05  # Small floor for baseline learning

In [ ]:
# 2. Apply improved triage score to cleaned text
df["triage_score"] = df["text"].apply(compute_triage_score)

In [ ]:
# 3. Prepare data for regression
from sklearn.model_selection import train_test_split
from datasets import Dataset

# Create base triage DataFrame
triage_df = df[["text", "triage_score"]].rename(columns={"text": "text", "triage_score": "label"})

# Identify high-risk rows and upweight them by repeating more
high_risk_df = triage_df[triage_df["label"] >= 0.75]
weighted_high_risk_df = pd.concat([high_risk_df]*5, ignore_index=True)  # repeat 5 times

# Combine with original data
augmented_full_df = pd.concat([triage_df, weighted_high_risk_df], ignore_index=True)
augmented_full_df = augmented_full_df.sample(frac=1.0, random_state=42)

In [ ]:
# 4. Train-Test Split
train_df, test_df = train_test_split(augmented_full_df, test_size=0.2, random_state=42)
train_dataset = Dataset.from_pandas(train_df.reset_index(drop=True))
test_dataset = Dataset.from_pandas(test_df.reset_index(drop=True))

In [ ]:
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")

def tokenize_fn(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=128)

train_dataset = train_dataset.map(tokenize_fn, batched=True)
test_dataset = test_dataset.map(tokenize_fn, batched=True)


Map:   0%|          | 0/9149 [00:00<?, ? examples/s]

Map:   0%|          | 0/2288 [00:00<?, ? examples/s]

In [ ]:
config = RobertaConfig(
    vocab_size=tokenizer.vocab_size,
    hidden_size=512,  # Reduce size for stability
    num_attention_heads=8,
    num_hidden_layers=4,
    intermediate_size=2048,
    max_position_embeddings=514,
    num_labels=1,
    problem_type="regression"
)

model_k = RobertaForSequenceClassification(config)


In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
from sklearn.metrics import mean_squared_error
import numpy as np

# Define training arguments
training_args = TrainingArguments(
    output_dir="./roberta-scratch-regression",
    num_train_epochs=6,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=5e-4,  # slightly higher since model is from scratch
    weight_decay=0.01,
    logging_dir="./logs",
)

# MSE as evaluation metric
def compute_metrics(eval_pred):
    preds, labels = eval_pred
    preds = preds.squeeze()
    return {"mse": mean_squared_error(labels, preds)}

trainer = Trainer(
    model=model_k,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer),
    compute_metrics=compute_metrics
)


/tmp/ipython-input-55-1650713509.py:22: FutureWarning:

`tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.



In [ ]:
trainer.train()
trainer.save_model("./roberta-scratch-regression")
tokenizer.save_pretrained("./roberta-scratch-regression")


Step,Training Loss
500,0.155900
1000,0.156200
1500,0.155300
2000,0.152200
2500,0.151800
3000,0.153700
3500,0.153600
4000,0.146800
4500,0.151700
5000,0.144900


('./roberta-scratch-regression/tokenizer_config.json',
 './roberta-scratch-regression/special_tokens_map.json',
 './roberta-scratch-regression/vocab.json',
 './roberta-scratch-regression/merges.txt',
 './roberta-scratch-regression/added_tokens.json',
 './roberta-scratch-regression/tokenizer.json')

In [ ]:
# Evaluate using Trainer's built-in evaluation
eval_results = trainer.evaluate(eval_dataset=test_dataset)
print("📊 Evaluation Metrics (on test set):")
for k, v in eval_results.items():
    print(f"{k}: {v:.4f}")


📊 Evaluation Metrics (on test set):
eval_loss: 0.1357
eval_mse: 0.1357
eval_runtime: 3.0109
eval_samples_per_second: 759.9050
eval_steps_per_second: 47.4940
epoch: 6.0000


In [ ]:
# Get predictions
predictions = trainer.predict(test_dataset)
preds = predictions.predictions.squeeze()
labels = predictions.label_ids.squeeze()


In [ ]:
# Create DataFrame for visualization
results_df_bert = pd.DataFrame({
    "True Score": labels,
    "Predicted Score": preds
})


In [ ]:
import plotly.express as px

fig = px.scatter(
    results_df_bert,
    x="True Score",
    y="Predicted Score",
    trendline="ols",
    title="📉 Triage Score: True vs Predicted",
    template="plotly_dark",
    color_discrete_sequence=["cyan"]
)

fig.update_layout(
    paper_bgcolor='black',
    plot_bgcolor='black',
    font=dict(color='white'),
    title_font=dict(size=20),
    xaxis_title="True Triage Score",
    yaxis_title="Predicted Triage Score"
)

fig.show()


In [ ]:
id_to_label = {
    0: "Stress",
    1: "Depression",
    2: "Bipolar disorder",
    3: "Personality disorder",
    4: "Anxiety"
}


In [ ]:
import torch
import numpy as np

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_r.to(device)
model_k.to(device)

def predict_condition_and_triage(texts):
    # Tokenize once
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Predict classification
    model_r.eval()
    with torch.no_grad():
        class_logits = model_r(**inputs).logits
        class_preds = class_logits.argmax(dim=1).tolist()
        class_labels = [id_to_label[p] for p in class_preds]

    # Predict triage score
    model_k.eval()
    with torch.no_grad():
        triage_scores = model_k(**inputs).logits.squeeze().cpu().numpy()
        triage_scores = np.clip(triage_scores, 0, 1)  # Ensure in [0, 1]

    # Output
    for text, label, score in zip(texts, class_labels, triage_scores):
        print(f"📝 Input:\n{text}")
        print(f"🔍 Predicted Condition: {label}")
        print(f"🚨 Triage Score: {score:.2f}")
        print("-" * 60)


In [ ]:
sample_posts = [
    "I don’t want to be here anymore. Everything hurts and I’m exhausted.",
    "Lately, I’ve been thinking about ending it all. I just want the pain to stop.",
    "I’m scared of myself. I’ve been making plans to take my own life.",
    "Nothing feels real. I keep thinking the world would be better without me.",
]
predict_condition_and_triage(sample_posts)

📝 Input:
I don’t want to be here anymore. Everything hurts and I’m exhausted.
🔍 Predicted Condition: Depression
🚨 Triage Score: 0.53
------------------------------------------------------------
📝 Input:
Lately, I’ve been thinking about ending it all. I just want the pain to stop.
🔍 Predicted Condition: Stress
🚨 Triage Score: 0.53
------------------------------------------------------------
📝 Input:
I’m scared of myself. I’ve been making plans to take my own life.
🔍 Predicted Condition: Depression
🚨 Triage Score: 0.53
------------------------------------------------------------
📝 Input:
Nothing feels real. I keep thinking the world would be better without me.
🔍 Predicted Condition: Depression
🚨 Triage Score: 0.53
------------------------------------------------------------


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import plotly.graph_objects as go
import numpy as np

# Calculate metrics
true_scores = results_df_bert["True Score"]
pred_scores = results_df_bert["Predicted Score"]

mae = mean_absolute_error(true_scores, pred_scores)
mse = mean_squared_error(true_scores, pred_scores)
rmse = np.sqrt(mse)
r2 = r2_score(true_scores, pred_scores)

# Create dataframe
metrics_table = pd.DataFrame({
    "Metric": ["Mean Absolute Error", "Mean Squared Error", "Root Mean Squared Error", "R² Score"],
    "Value": [mae, mse, rmse, r2]
})

# Plotly Table
fig = go.Figure(data=[go.Table(
    header=dict(
        values=["📏 Metric", "🔢 Value"],
        fill_color="darkslategray",
        font=dict(color='white', size=14),
        align="left"
    ),
    cells=dict(
        values=[metrics_table.Metric, [f"{v:.4f}" for v in metrics_table.Value]],
        fill_color="black",
        font=dict(color='white', size=12),
        align="left"
    )
)])

fig.update_layout(
    title="📋 Regression Evaluation Metrics (Triage Score)",
    paper_bgcolor="black",
    plot_bgcolor="black",
    title_font=dict(size=20, color="white"),
    height=350
)

fig.show()


### ❗ Justification for Poor Performance When Training RoBERTa from Scratch

Training RoBERTa from scratch — without any pretrained weights — proved ineffective for our triage score regression task. This is primarily because models like RoBERTa are designed to be fine-tuned after large-scale pretraining on billions of tokens. Without this foundation, the model starts from random weights and lacks any understanding of language structure or semantics. Given our dataset is relatively small, domain-specific (mental health context), and imbalanced (fewer high-risk samples), it simply does not provide enough signal for the model to learn meaningful representations from scratch. Furthermore, regression is inherently more sensitive than classification; it requires the model to learn fine-grained distinctions between inputs, which is extremely difficult without prior contextual embeddings. The resulting metrics — such as a near-zero or negative R² score and high MAE/RMSE — confirmed that the model essentially failed to learn. This reinforces that for nuanced tasks like mental health triage scoring, leveraging pretrained language models is not just helpful, but essential.


In [1]:
!jupyter nbconvert --to python "/content/roberta_classification.ipynb"

[NbConvertApp] Converting notebook /content/roberta_classification.ipynb to python
[NbConvertApp] Writing 19093 bytes to /content/roberta_classification.py
